In [ ]:
using Pkg
# Activate environment that has ProgressiveHedging installed
home = homedir()
Pkg.activate(joinpath(home,".julia/environments/v1.2"))
Pkg.status()

In [ ]:
using Distributed
const WORKERS = 1 # Change to > 1 to use parallel
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
    # Make sure these workers also have an environment with PH installed
    @everywhere using Pkg
    for w in workers()
        @spawnat(w, Pkg.activate(joinpath(home,".julia/environments/v1.2")))
    end
end

@everywhere using ProgressiveHedging
const PH = ProgressiveHedging
using Ipopt

using StructJuMP
@everywhere using JuMP

### Model Construction Function Definition

In [ ]:
function variable_dict()
    first_stage_vars = ["x[1]", "x[2]", "x[3]"]
    second_stage_vars = ["y"]
    third_stage_vars = ["z[1]", "z[2]"]

    var_dict = Dict(1=>first_stage_vars,
                    2=>second_stage_vars,
                    3=>third_stage_vars)
    return var_dict
end

@everywhere function create_model(scenario_id::Int, model::M,
        additional_arg::String; key_word_arg::String="Default key word argument", kwargs...
        )::M where M <: JuMP.AbstractModel
    
    println(additional_arg)
    println(key_word_arg)
    
    c = [1.0, 10.0, 0.01]
    d = 7.0
    a = 16.0

    α = 1.0
    β = 1.0
    γ = 1.0
    δ = 1.0
    ϵ = 1.0

    s1 = 8.0
    s2 = 4.0
    s11 = 9.0
    s12 = 16.0
    s21 = 5.0
    s22 = 18.0
    
    JuMP.@variable(model, x[1:3] >= 0.0)
    JuMP.@constraint(model, x[3] <= 1.0)
    obj = zero(JuMP.GenericQuadExpr{Float64,JuMP.VariableRef})
    JuMP.add_to_expression!(obj, sum(c.*x))

    # Second stage
    if scenario_id < 2
        JuMP.@variable(model, y >= 0.0)
        JuMP.@constraint(model, α*sum(x) + β*y >= s1)
        JuMP.add_to_expression!(obj, d*y)
    else
        JuMP.@variable(model, y >= 0.0)
        JuMP.@constraint(model, α*sum(x) + β*y >= s2)
        JuMP.add_to_expression!(obj, d*y)
    end

    # Third stage
    if scenario_id == 0
        JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s11)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))
        
    elseif scenario_id == 1
        JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s12)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))

    elseif scenario_id == 2
        JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s21)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))

    else
        JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s22)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))
    end

    JuMP.@objective(model, Min, obj)
    
    return model
end
;

### Build Scenario Tree Using StructJuMP
We can build the scenario tree using StructJuMP and convert it to the appropriate PH type by using the function PH.build_scenario_tree. Note that we do NOT need to add variables, constraints or objectives to the StructJuMP models to use this functionality.  The only things that are used are the parents and probabilities settings.  

Warning: PH code actually modifies the scenario tree so either a deep copy of the tree should be passed to the PH code or a new scenario tree should be constructed with each call to solve.

In [ ]:
nscen = 4
nbranch = 2

# First stage
root_model = StructuredModel(num_scenarios=nbranch)

# Second stage
mid_model_1 = StructuredModel(parent=root_model, id=1, num_scenarios=nbranch, prob=0.5)
mid_model_2 = StructuredModel(parent=root_model, id=2, num_scenarios=nbranch, prob=0.5)

# Third stage
leaf_11 = StructuredModel(parent=mid_model_1, id=11, prob=0.75)
leaf_12 = StructuredModel(parent=mid_model_1, id=12, prob=0.25)
leaf_21 = StructuredModel(parent=mid_model_2, id=21, prob=0.75)
leaf_22 = StructuredModel(parent=mid_model_2, id=22, prob=0.25)
;

In [ ]:
@time (n, err, obj, soln, phd) = PH.solve(PH.build_scenario_tree(root_model),
                                          create_model, variable_dict(),
                                          with_optimizer(Ipopt.Optimizer, print_level=0, tol=1e-12),
                                          25.0, "This is a required argument!",
                                          atol=1e-8, max_iter=500, report=false,
                                          unused_kwarg="I am unused!!")
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

In [ ]:
soln

In [ ]:
PH.retrieve_no_hats(phd)

In [ ]:
PH.retrieve_w(phd)

### Build Scenario Tree Directly
Build the tree ourselves.  Note that you must call add_leaf when adding a leaf node instead of add_node.  Failing to do so will lead to unusual behavior.

In [ ]:
function build_scen_tree()

    probs = [0.5*0.75, 0.5*0.25, 0.5*0.75, 0.5*0.25]
    
    tree = PH.ScenarioTree()
    
    for k in 1:2
        node2 = PH.add_node(tree, tree.root)
        for l in 1:2
            PH.add_leaf(tree, node2, probs[(k-1)*2 + l])
        end
    end
    return tree
end
;

In [ ]:
@time (n, err, obj, soln, phd) = PH.solve(build_scen_tree(),
                                          create_model, variable_dict(),
                                          with_optimizer(Ipopt.Optimizer, print_level=0, tol=1e-12), 
                                          25.0, "Passed to constructor.",
                                          atol=1e-8, max_iter=500, report=true,
                                          key_word_arg="We changed this key word arg!")
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

In [ ]:
soln

In [ ]:
PH.retrieve_no_hats(phd)

In [ ]:
PH.retrieve_w(phd)